* `ndarray.view(dtype)`: Shows reinterpretation of the array in the format of `dtype`. (Documentation)
    * E.g., if `x = np.array(9.4)`, then `x.view(np.int64)` reinterpret the chunk of bit for 9.4, a 64-digit binary number, as an integer. In words, it says to NumPy "View `x` as integer."
* `numpy.binary_repr`: Return the binary representation of the input number as a string. (Documentation)
    * If `width=64` is passed, the output consists of all 64 bits, including the first consecutive 0's.
* `numpy.base_repr(x.view(np.int64), base=16)`: Return the hexadecimal representation of the input, in this case, `x.view(np.int64)`. (Documentation)

In [ ]:
import numpy as np
x = np.array(9.4)
print(f"{'Usual print of '              :>27}{x}{':'            :>13}{x}")
print(f"{'Float interpretation of '     :>27}{x}{':'            :>13}{x.view(np.float64)}")
print(f"{'Integer interpretation of '   :>27}{x}{' (base  2) :' :>13}{np.binary_repr(x.view(np.int64), width=64)}")
print(f"{'Integer interpretation of '   :>27}{x}{' (base 10) :' :>13}{x.view(np.int64)}")
print(f"{'Integer interpretation of '   :>27}{x}{' (base 16) :' :>13}{np.base_repr(x.view(np.int64), base=16)}")

In [ ]:
# smallest positive number: 
#   zero exponent, only one 1 in the last bit of the mantissa
x = np.array(1, dtype=np.int64)
y = x
print(f"{'Ex0   (bit): ':>13}{np.binary_repr(y, width=64)}")
print(f"{'(float): '    :>13}{y.view(np.float64)}")

# flip the sign bit
# 1 is at the first (sign) and the last bits (mantissa)
y = x + np.left_shift(x, 63)
print(f"{'Ex1   (bit): ':>13}{np.binary_repr(y, width=64)}")
print(f"{'(float): '    :>13}{y.view(np.float64)}")

# 0.1*2^(-1022) = 1.0*2^(-1023)
# 1 is at the leftmost bit of the mantissa (0 exponent; subnormal)
y = np.left_shift(x, 51)
print(f"{'Ex2   (bit): ':>13}{np.binary_repr(y, width=64)}")
print(f"{'(float): '    :>13}{y.view(np.float64)}")

# 1.0*2^(-1022)
# 1 is at the rightmost bit of the exponent (1 exponent; normalized)
y = np.left_shift(x, 52)
print(f"{'Ex3   (bit): ':>13}{np.binary_repr(y, width=64)}")
print(f"{'(float): '    :>13}{y.view(np.float64)}")

### Additioon of floating point numbers
* Algorithm
    * Given two numbers, line up the decimal places, add the two numbers, store result
* Actual addition can be conducted in higher precision than 52 bits, but the result is rounded to 52 bits of mantissa

In [ ]:
x = 1.0
y = 2**(-53)
z = 2**(-52)

print(f"{'x: ' :>5}{x}")
print(f"{'y: ' :>5}{y}")
print(f"{'z: ' :>5}{z}")
print(f"{'x+y: ' :>5}{x+y}")
print(f"{'x+z: ' :>5}{x+z}")

This method of addition can lead to roundings and truncations that give surprising results

In [ ]:
x = 9.4
y = x - 9.4
z = x - 9.0
z = z - 0.4

print(f"{'x: ' :>5}{x}")
print(f"{'y: ' :>5}{y}")
print(f"{'z: ' :>5}{z}")

Numbers smaller than $\epsilon_{mach} (= 2^{-52})$ aren't negligible in the IEEE model. If they're representative in the model, computations are accurate as long as the result isn't added to a number of unit size (something relatively larger)

In [ ]:
r = 1e-20 # << 2^(-52)=2.220446049250313e-16
x = 9.4 * r
y = x - (9.4 * r)
z = x - (9.0 * r)
z = z - (0.4 * r)

print(f"{'x: ' :>5}{x}")
print(f"{'y: ' :>5}{y}")
# The computing error is not of order of machine epsilon,
# but it is compatible with (x * e_mach)
print(f"{'z: ' :>5}{z}")

### Example from Textbook
Suppose we compute $E_1$, which is equivalent to $E_2 \\$
$E_1 = \frac{1-\text{cos}x}{sin^2x}$ and $E_2 = \frac{1}{1+\text{cos}x} \\$
Implement a series of evaluations of $E_1, E_2$ at $x = 1/10^{j}$ with $j = 0,1,2,\dots,14$. Which one is more credible and why?

In [ ]:
import pandas as pd

j = np.arange(15)
x = 10.**(-j)
E1 = lambda x: (1- np.cos(x))/(np.sin(x)*np.sin(x))
E2 = lambda x: 1/(1+np.cos(x))

df = pd.DataFrame({'x': x, 'E1': E1(x), 'E2': E2(x)})
print(df)